# DPS + STAC integration: HLS Cloud Free Temporal Mosaic Example

This notebook demonstrates how DPS algorithms that produce STAC metadata that describe the output files can produce rich STAC collections that are easily queryable and can be visualized using the titiler-pgstac deployment that is attached to the MAAP User STAAC.

### Algorithm
The HLS Cloud Free Temporal Mosaic algorithm is hosted here: https://github.com/MAAP-Project/hls-cloud-free-temporal-mosaic

It is used to produce COGs that contain the median reflectance value from temporal mosaics of HLS imagery (HLSL30_2.0 and HLSS30_2.0) with clouds and cloud shadows masked out.

The algorithm script generates a STAC item that includes the output COGs as assets and adds it to a catalog.json file which gets written to the DPS output folder (which gets uploaded to S3).

### STAC ingestion pipeline
When the catalog.json file lands in the maap-ops-workspace S3 bucket, it triggers a STAC ingestion pipeline that will find the STAC items in S3, validate them, and add them to a STAC collection associated with the user who submitted the job, the algorithm + version that was used, and the job tag.

## DPS job setup
The next few cells show how the algorithm was registered and how to submit a bunch of DPS jobs that will produce COGs and STAC metadata.

In [1]:
from datetime import datetime, timedelta, UTC

from maap.maap import MAAP

maap = MAAP()

In [56]:
# maap.register_algorithm_from_yaml_file("/projects/algorithm-configs/hls-cloud-free-temporal-mosaic.yml").text

'{"code": 200, "message": {"id": "1e4c6b3a4d15da4abe7c0e0e87e4a1d84ae42f48", "short_id": "1e4c6b3a", "created_at": "2025-11-06T13:07:41.000+00:00", "parent_ids": ["bc6081f51d26b8624e5053bfbd8737e8960ddc7b"], "title": "Registering algorithm: HLSCloudFreeTemporalMosaic", "message": "Registering algorithm: HLSCloudFreeTemporalMosaic", "author_name": "root", "author_email": "root@971fce4a40be", "authored_date": "2025-11-06T13:07:41.000+00:00", "committer_name": "root", "committer_email": "root@971fce4a40be", "committed_date": "2025-11-06T13:07:41.000+00:00", "trailers": {}, "extended_trailers": {}, "web_url": "https://repo.maap-project.org/root/register-job-hysds-v4/-/commit/1e4c6b3a4d15da4abe7c0e0e87e4a1d84ae42f48", "stats": {"additions": 0, "deletions": 0, "total": 0}, "status": "pending", "project_id": 3, "last_pipeline": {"id": 16347, "iid": 2788, "project_id": 3, "sha": "1e4c6b3a4d15da4abe7c0e0e87e4a1d84ae42f48", "ref": "hysds-v5", "status": "pending", "source": "push", "created_at": 

### Generate a regular grid to use for STAC item footprint

One of the inputs to the algorithm a set of bounding box coordinates (in a projected CRS). The bounding box coordinates will define the extent of the output COGs (and the STAC item).

This code generates a set of 8192 x 8192 (pixels) extents which are squares that are ~250 km on a side. 

In [60]:
full_bbox = (-105000, 2264000, 566000, 2937000)

bboxes = []
resolution = 30
grid_len_pixels = 8192
grid_len_meters = resolution * grid_len_pixels
xmin_orig, ymin_orig = full_bbox[:2]

xmin_start = xmin_orig - xmin_orig % grid_len_meters
ymin_start = ymin_orig - ymin_orig % grid_len_meters

xmin = xmin_start

while xmin < full_bbox[2]:
    xmax = xmin + grid_len_meters

    ymin = ymin_start

    while ymin < full_bbox[3]:
        ymax = ymin + grid_len_meters

        bboxes.append((xmin, ymin, xmax, ymax))

        ymin = ymax

    xmin = xmax

bboxes

[(-245760, 2211840, 0, 2457600),
 (-245760, 2457600, 0, 2703360),
 (-245760, 2703360, 0, 2949120),
 (0, 2211840, 245760, 2457600),
 (0, 2457600, 245760, 2703360),
 (0, 2703360, 245760, 2949120),
 (245760, 2211840, 491520, 2457600),
 (245760, 2457600, 491520, 2703360),
 (245760, 2703360, 491520, 2949120),
 (491520, 2211840, 737280, 2457600),
 (491520, 2457600, 737280, 2703360),
 (491520, 2703360, 737280, 2949120)]

### Generate a set of year/month combinations

The algorithm is useful for creating cloud-free composite images for periods of time (e.g. monthly). This cell generates the year/month combos we will use to define the temporal extent of each composite image.

In [80]:
year_months = [(year, month) for year in [2024] for month in [6, 7, 8, 9, 10]]
print(year_months)

datetime_fmt = "%Y-%m-%dT%H:%M:%SZ"

[(2024, 6), (2024, 7), (2024, 8), (2024, 9), (2024, 10)]


### Submit jobs to DPS
Submit a job for each bbox + month combination to generate a few months worth of cloud-free imagery for each AOI.

In [77]:
for bbox in bboxes:
    for year, month in year_months:
        start_datetime = datetime(year, month, 1, tzinfo=UTC)
        end_datetime = datetime(year, month + 1, 1, tzinfo=UTC) - timedelta(seconds=1)

        maap.submitJob(
            algo_id="HLSCloudFreeTemporalMosaic",
            version="v0.1.1",
            identifier="demo",
            queue="maap-dps-worker-16gb",
            start_datetime=start_datetime.strftime(datetime_fmt),
            end_datetime=end_datetime.strftime(datetime_fmt),
            bbox=" ".join(str(x) for x in bbox),
            crs="epsg:5070",
        )

## Examine the results via the MAAP User STAC!

Each job takes ~20 minutes to finish. The product of these jobs is 60 AOI x month HLS composite images (3 bands each). You could trawl through the maap-ops-workspace bucket to find and use the results, but since we generated STAC metadata for the output of each job, we can use the MAAP User STAAC instead.

The STAC item ingestion process is asynchronous with the DPS job progress so there is a lag (<5 minutes) between when the outputs are uploaded to S3 and when the STAC metadata is available in the MAAP User STAC.

### Querying the MAAP User STAC with pystac-client
The MAAP User STAC is not fully released or documented yet, but it is working.

In [3]:
from pystac_client import Client

MAAP_USER_STAC_API_URL = "https://mxyjvl46y3.execute-api.us-west-2.amazonaws.com"

client = Client.open(MAAP_USER_STAC_API_URL)

for collection in client.collection_search(q="henrydevseed").collections():
    print(collection.id)

henrydevseed__HLSCloudFreeTemporalMosaic__v0.1.1__demo
henrydevseed__HLSCloudFreeTemporalMosaic__v0.1.1__mic-check
henrydevseed__STACOutputDemo__main__test-20251103


The collection we want is `henrydevseed__HLSCloudFreeTemporalMosaic__v0.1.1__demo` based on the input parameters for the jobs we submitted.

You can query the items using pystac-client like this:

In [5]:
collection_id = "henrydevseed__HLSCloudFreeTemporalMosaic__v0.1.1__demo"

search = client.search(
    collections=collection_id,
    datetime="2024-06-01",
)

items = search.item_collection()
items

These STAC items could be loaded into xarray using a tool like `odc-stac`!

### Visualizing the results with titiler-pgstac
The MAAP User STAC is connected to a titiler-pgstac deployment which can render mosaics on-the-fly from any collection.

In [14]:
import httpx
from folium import LayerControl, Map, TileLayer

MAAP_USER_STAC_TITILER_API_URL = (
    "https://ansdoe39vf.execute-api.us-west-2.amazonaws.com"
)

tilejsons = {
    dt: httpx.get(
        f"{MAAP_USER_STAC_TITILER_API_URL}/collections/{collection_id}/WebMercatorQuad/tilejson.json",
        params={
            "assets": ["red", "green", "blue"],
            "color_formula": "Gamma RGB 3.5 Saturation 1.2 Sigmoidal RGB 15 0.35",
            "datetime": dt.isoformat(),
        },
    ).json()
    for dt in [
        datetime(year, month, 1) for year in [2024] for month in [6, 7, 8, 9, 10]
    ]
}

m = Map(location=[46.3, -94.2], zoom_start=6, min_zoom=5)

for i, (dt, tilejson) in enumerate(tilejsons.items()):
    label = dt.strftime("%Y-%m")
    TileLayer(
        tiles=tilejson["tiles"][0],
        attr="NASA MAAP",
        overlay=True,
        name=label,
        show=(i == 0),
    ).add_to(m)

LayerControl(collapsed=False).add_to(m)

m